In [2]:
anki_path = r'C:\Users\hui\AppData\Local\Programs\Anki\anki.exe'

nested_bullet_points_prompt = '''Summarize the following text in a nested bullet point format so the output can be pasted into Workflowy which requires the following format:

- Title
    - Subtitle
        - Sub-subtitle
            - Key point
            - Key point

Each level should be indented using 4 spaces for clarity.
Begin all bullet points with a dash followed by a single space.
Avoid using hashtags or any special characters such as asterisks or numbering.
Do not add any additional commentary or explanations. Only present the summarized bullet points.'''

anki_cloze_cards_prompt = '''Generate Anki-style cloze deletion flashcards from every key point in the passage below.
- Do not miss any key points
- Use the format `{{c1::hidden text}}`
- Turn key facts, terms, or concepts into cloze deletions
- Each card should be on a new line
- Do not add any additional commentary or explanations.
- Only present the cloze deletion cards.
- Example Outputs (These are examples. Do NOT generate the following):
The {{c1::mitochondrion}} is known as the powerhouse of the cell.
It generates most of the cell's supply of {{c1::ATP}}.
ATP is used as a source of {{c1::chemical energy}}.'''

In [3]:
import os
import time
import psutil
import requests
import webbrowser
import subprocess
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
DEEPSEEK_KEY = os.getenv('DEEPSEEK_KEY')

client = OpenAI(
    api_key=DEEPSEEK_KEY,
    base_url="https://api.deepseek.com"
)

def get_llm_response(user_input, system_input=None):
    messages = [{ "role": "user", "content": user_input}]
    if system_input:
        messages = [
            {"role": "system", "content": system_input},
        ] + messages
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        stream=False
    )

    return response.choices[0].message.content


def get_nested_bullet_points(user_input):
    print('Generating nested bullet points...')
    return get_llm_response(user_input, nested_bullet_points_prompt)


def get_anki_cloze_cards(user_input):
    print('Generating Anki cloze cards...')
    str_output = get_llm_response(user_input, anki_cloze_cards_prompt)
    str_output = str_output.replace('\n\n', '\n')
    cloze_cards = [card.strip() for card in str_output.split('\n')]
    return cloze_cards

def start_anki():
    print('Starting Anki...')
    subprocess.Popen(anki_path)
    time.sleep(5)


def create_deck(deck_name):
    print(f'Creating deck: {deck_name}')
    requests.post('http://localhost:8765', json={
        'action': 'createDeck',
        'version': 6,
        'params': {
            'deck': deck_name
        }
    })


def get_title_body_lists(path):

    with open(path, 'r', errors='ignore') as f:
        contents = f.read()

    title_body_lists = []
    contents = contents.split('~~~~~')
    zfill_len = len(str(len(contents)))
    should_enumerate = len(contents) > 1
    for i, content in enumerate(contents):
        title, *body = content.strip().split('\n')
        try:
            title = title.split('. ')[1]
        except:
            pass
        if should_enumerate:
            title = f'{str(i+1).zfill(zfill_len)} {title}'
        body = '\n'.join(body)
        title_body_lists.append((title, body))

    return title_body_lists


def add_cloze_cards(deck_name, cloze_cards):
    print('Adding cloze cards to deck:')
    for cloze_card in cloze_cards:
        note = {
            'deckName': deck_name,
            'modelName': 'Cloze',
            'fields': {
                'Text': cloze_card
            },
            'options': {
                'allowDuplicate': True
            },
            'tags': []
        }

        payload = {
            'action': 'addNote',
            'version': 6,
            'params': {
                'note': note
            }
        }

        try:
            response = requests.post('http://localhost:8765', json=payload).json()
            if 'error' in response and response['error']:
                print(f'Failed to add card: {cloze_card}\nError: {response["error"]}')
            else:
                print(f'Added card: {cloze_card}')
        except Exception as e:
            print(f'Exception occurred: {e}')


def sync_anki():
    print('Syncing Anki...')
    requests.post("http://localhost:8765", json={
        "action": "sync",
        "version": 6
    })


def stop_anki():
    for proc in psutil.process_iter(['name']):
        if proc.info['name'] and 'anki' in proc.info['name']:
            print(f'''Terminating {proc.info['name']}''')
            proc.terminate()


def open_ankiweb():
    print('Opening AnkiWeb...')
    webbrowser.open('https://ankiweb.net')


def create_cloze_cards_and_sync(suffix, path='anki.txt'):

    title_body_lists = get_title_body_lists(path)
    response = input('\n'.join([t for t, _ in title_body_lists]) + '\nWould you like to create these decks? (y/n) ')
    if response.strip().lower() != 'y':
        return

    start_anki()
    for title, passage in title_body_lists:
        if suffix and not suffix.endswith('::'):
            suffix += '::'
        deck_name = suffix + title
        create_deck(deck_name)
        cloze_cards = get_anki_cloze_cards(passage)
        add_cloze_cards(deck_name, cloze_cards)

    sync_anki()
    # stop_anki()
    # open_ankiweb()

In [4]:
create_cloze_cards_and_sync('AllTrue::Walkthroughs::4. Compliance')

Starting Anki...
Creating deck: AllTrue::Walkthroughs::4. Compliance1 Part 1
Generating Anki cloze cards...
Adding cloze cards to deck:
Failed to add card: Here are the Anki-style cloze deletion flashcards based on the key points in the passage:
Error: cannot create note for unknown reason
Added card: 1. The session will cover {{c1::pentest categories}} in a separate session.
Added card: 2. Today's session is substantial due to the complexity of {{c1::TPRM}} and {{c1::compliance audits}}.
Added card: 3. System slowness is observed in modules due to the large scale of {{c1::TPRM}} and {{c1::compliance audits}}.
Added card: 4. An example of task creation in the TPRM module is {{c1::Copilot task creation in Asana}}.
Added card: 5. The current solution for notification issues is to instruct customers to adjust {{c1::notification settings upon invitation}}.
Added card: 6. Only {{c1::customers}} can control their notification settings, with no permanent solution yet.
Added card: 7. The impac